In [89]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import warnings
warnings.simplefilter("ignore")

In [90]:
posts = pd.read_csv("Posts.csv")
tweets = posts['Text'].values
y_data = posts['Ideation'].values
posts['Text'] = posts['Text'].str.lower()

posts['Text'] = posts['Text'].str.replace('(#(\w)+)', '')
posts['Text'] = posts['Text'].str.replace('(@(\w)+)', '')

posts['Text'] = posts['Text'].str.replace('&', 'and')
posts['Text'] = posts['Text'].str.replace('a-z0-9\.\?\:\. |', '')
posts['Text'] = posts['Text'].str.replace('<[^<>]+>', '')
posts['Text'] = posts['Text'].str.replace('[0-9]+', 'number')
posts['Text'] = posts['Text'].str.replace('(http|https)://[^\s]*', '')
posts['Text'] = posts['Text'].str.replace('[\.]+', '.')
posts['Text'] = posts['Text'].str.replace('[\,]+', ',')
posts['Text'] = posts['Text'].str.replace('[\?]+', '?')
posts['Text'] = posts['Text'].str.replace('[\!]+', '!')

In [91]:
x_train, x_test, y_train, y_test = train_test_split(tweets, y_data, test_size = 0.25, random_state = 42)

In [92]:
x_train, x_test = train_test_split(posts, test_size=0.25, random_state=42)
x_train.head()
model = LogisticRegression()
model.fit(X_train, Y_train)

training_accuracy = model.score(X_train, Y_train)
print("Training Accuracy: ", training_accuracy)

testing_accuracy = model.score(X_test, Y_test)
print("Testing Accuracy: ", testing_accuracy)

Training Accuracy:  0.5561172901921132
Testing Accuracy:  0.4727272727272727


In [93]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [94]:
x_data = posts['Text'].values
y_data = posts['Ideation'].values
x_data
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.25, random_state = 42)

In [95]:
x_train

array(['imagine being a cop id kill myself',
       "mainland india, you can't do shit for us ne indigenous folks, but at the very least, can you stop this vulgar display of sheer neglect and callousness by the govt. \n\noutrage! if that means we all die of covid-number so be it. this country is not worth living in if we allow this.",
       'no fucking way, i’m actually going to commit suicide.',
       'mentally and emotionally drained. i wish i can just sleep forever and never wake up. ',
       'i wanna get choked, i just wanna die',
       'i just slept for number hours. i’m tired of living.',
       'i think i will go end my life from loosing everything',
       'i’m about to cry. i really don’t want to be here',
       'i’m so unwanted and unloveable. i just wanna die',
       'i want to kill myself not because i want to end my life. i just want to end my endless pain.',
       'what if suicide is the only answer to your problem？',
       'wanna fucking die rn. i just wanna die'

In [96]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(x_train)

X_train = tokenizer.texts_to_sequences(x_train)
X_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

In [97]:
maxlen = 50
X_train = pad_sequences(X_train, padding = 'post', maxlen = maxlen)
X_test = pad_sequences(X_test, padding = 'post', maxlen = maxlen)
encoder = OneHotEncoder(sparse = False)

y_train = y_train.reshape((len(y_train), 1))
y_train = encoder.fit_transform(y_train)

y_test = y_test.reshape((len(y_test), 1))
y_test = encoder.fit_transform(y_test)

In [98]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    with open(filepath, encoding = 'utf8') as f:
        for line in f:
            values = line.split()
            if values[0] in word_index:
                idx = word_index[values[0]]
                embedding_matrix[idx] = np.asarray(values[1:embedding_dim+1], 'float32')
                
    return embedding_matrix

In [99]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('glove.twitter.27B.50d.txt', 
                                           tokenizer.word_index, 
                                           embedding_dim)

In [100]:
model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(32, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(pool_size = 3, strides = 1, padding = 'valid'))
model.add(layers.Dropout(0.3))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation = 'softmax'))

In [101]:
model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(48, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(3, 1, padding = 'valid'))
model.add(layers.Dropout(0.3))
model.add(layers.GRU(48, return_sequences = True))
#model.add(layers.GRU(12, return_sequences = False))
#model.add(layers.Dropout(0.3))
model.add(layers.SimpleRNN(12))
model.add(layers.Dense(3, activation = 'softmax'))

In [102]:
model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(32, 4, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(32, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(32, 2, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation = 'softmax'))

In [103]:
model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim,
                           weights = [embedding_matrix],
                           input_length = maxlen,
                           trainable = True))
model.add(layers.Conv1D(32, 4, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(32, 3, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(32, 2, padding = 'valid', strides = 1, activation = 'relu'))
model.add(layers.MaxPool1D(2, 1))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation = 'softmax'))

In [105]:
model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs= 50,
                    batch_size= 52,
                    validation_data=(X_test, y_test)
                   )
loss = model.evaluate(X_train, y_train, verbose=False) 
accuracy = model.evaluate(X_train, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))


Train on 989 samples, validate on 330 samples
Epoch 1/50
989/989 [==============================] - 66s 66ms/step - loss: 0.7986 - acc: 0.6229 - val_loss: 0.8417 - val_acc: 0.6061
Epoch 2/50
989/989 [==============================] - 59s 59ms/step - loss: 0.7405 - acc: 0.6825 - val_loss: 0.8302 - val_acc: 0.6000
Epoch 3/50
989/989 [==============================] - 62s 63ms/step - loss: 0.7096 - acc: 0.6886 - val_loss: 0.8110 - val_acc: 0.6485
Epoch 4/50
989/989 [==============================] - 64s 65ms/step - loss: 0.6786 - acc: 0.6946 - val_loss: 0.7944 - val_acc: 0.6485
Epoch 5/50
989/989 [==============================] - 65s 66ms/step - loss: 0.6290 - acc: 0.7300 - val_loss: 0.8039 - val_acc: 0.6091
Epoch 6/50
989/989 [==============================] - 65s 66ms/step - loss: 0.5574 - acc: 0.7765 - val_loss: 0.7930 - val_acc: 0.6485
Epoch 7/50
989/989 [==============================] - 55s 55ms/step - loss: 0.5030 - acc: 0.7958 - val_loss: 0.7900 - val_acc: 0.6485
Epoch 8/50
989/9

In [106]:
prediction_train = [i.argmax() for i in model.predict(X_train)]
prediction_test = [i.argmax() for i in model.predict(X_test)]

Y_train_hat = np.array(prediction_train)
Y_test_hat = np.array(prediction_test)
Y_train = np.array([i.argmax() for i in y_train])
Y_test = np.array([i.argmax() for i in y_test])

tp = sum((Y_test == Y_test_hat) & (Y_test == 2))
tn = sum((Y_test == Y_test_hat) & (Y_test != 2))
fp = sum((Y_test != Y_test_hat) & (Y_test_hat == 2))
fn = sum((Y_test != Y_test_hat) & (Y_test == 2))

precision = tp/(tp+fp)
recall = tp/(tp+fn)

f1 = 2 * precision * recall / (precision + recall)
precision, recall, f1

(0.7162162162162162, 0.6973684210526315, 0.7066666666666668)